In [1]:
# import datetime
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
from datlib.FRED import *
from datlib.plots import *
import pandas_datareader.data as web


#FRED.py
#. . . 
def bil_to_mil(series):
    return series* 10**3
# . . .
#fedProject.py
# . . .
data_codes  = {# Assets
               "Balance Sheet: Total Assets ($ Mil)": "WALCL",
               "Balance Sheet Securities, Prem-Disc, Repos, and Loans ($ Mil)": "WSRLL",
               "Balance Sheet: Securities Held Outright ($ Mil)": "WSHOSHO",
               ### breakdown of securities holdings ###
               "Balance Sheet: U.S. Treasuries Held Outright ($ Mil)":"WSHOTSL",
               "Balance Sheet: Federal Agency Debt Securities ($ Mil)" : "WSHOFADSL",
               "Balance Sheet: Mortgage-Backed Securities ($ Mil)": "WSHOMCB",
               # other forms of lending
               "Balance Sheet: Repos ($ Mil)": "WORAL",
               "Balance Sheet: Central Bank Liquidity Swaps ($ Mil)" : "SWPT",
               "Balance Sheet: Direct Lending ($ Mil)" : "WLCFLL",
               # unamortized value of securities held (due to changes in interest rates)
               "Balance Sheet: Unamortized Security Premiums ($ Mil)": "WUPSHO",
               # Liabilities
               "Balance Sheet: Total Liabilities ($ Mil)" : "WLTLECL",
               "Balance Sheet: Federal Reserve Notes Outstanding ($ Mil)" : "WLFN",
               "Balance Sheet: Reverse Repos ($ Mil)": "WLRRAL",
               ### Major share of deposits 
               "Balance Sheet: Deposits from Dep. Institutions ($ Mil)":"WLODLL",
               "Balance Sheet: U.S. Treasury General Account ($ Mil)": "WDTGAL",
               "Balance Sheet: Other Deposits ($ Mil)": "WOTHLB",
               "Balance Sheet: All Deposits ($ Mil)": "WLDLCL",
               # Capital
               "Balance Sheet: Total Capital": "WCTCL",
               # Interest Rates
               "Unemployment Rate": "UNRATE",
               "Nominal GDP ($ Bil)":"GDP",
               "Real GDP ($ Bil)":"GDPC1",
               "GDP Deflator":"GDPDEF",
               "CPI":"CPIAUCSL",
               "Core PCE":"PCEPILFE",
               "Private Investment":"GPDI",
               "Base: Total ($ Mil)": "BOGMBASE",
               "Base: Currency in Circulation ($ Bil)": "WCURCIR",
               "1 Month Treasury Rate (%)": "DGS1MO",
               "3 Month Treasury Rate (%)": "DGS3MO",               
               "1 Year Treasury Rate (%)": "DGS1",
               "2 Year Treasury Rate (%)": "DGS2",
               "10 Year Treasury Rate (%)": "DGS10",
               "30 Year Treasury Rate (%)": "DGS30",               
               "Effective Federal Funds Rate (%)": "DFF",
               "Federal Funds Target Rate (Pre-crisis)":"DFEDTAR",
               "Federal Funds Upper Target":"DFEDTARU",
               "Federal Funds Lower Target":"DFEDTARL",
               "Interest on Reserves (%)": "IOER",
               "VIX": "VIXCLS",
                "5 Year Forward Rate": "T5YIFR"
               }
inflation_target = .02
unemployment_target = .04
# Select start and end dates
start = datetime.datetime(2001, 12, 1)
end = datetime.datetime.today()
annual_div = {"Q":4,
             "W":52,
             "M":12}
# freq refers to data frequency. Choose "D", "W", "M", "Q", "A"
# a number may also be place in front of a letter. "2D" indicates
#       alternating days
if "data_gathered" not in locals():
    freq = "M"
    year = annual_div[freq]
    data = gather_data(data_codes, start, 
          end = end, freq = freq)
    data.fillna(0, inplace=True)
    for key in data.keys():
        data["Log " + key]= np.log(data[key])
    # Create new variables
    data_gathered = True

ticker = "^GSPC"
data["Base: Currency in Circulation ($ Mil)"] = data["Base: Currency in Circulation ($ Bil)"].mul(1000)
data["Base: Currency not in Circulation ($ Mil)"] = data["Base: Total ($ Mil)"].sub(data["Base: Currency in Circulation ($ Mil)"])
data["Currency in Circulation Growth Rate (%)"] = data["Base: Currency in Circulation ($ Mil)"].pct_change(year) * 100
data["% Currency not in Circulation"] = data["Base: Currency not in Circulation ($ Mil)"].div(data["Base: Total ($ Mil)"]) * 100
data["% Currency in Circulation"] = data["Base: Currency in Circulation ($ Mil)"].div(data["Base: Total ($ Mil)"]) * 100
data["Base: Total Growth Rate (%)"] = data["Base: Total ($ Mil)"]
data["Change % Currency not in Circulation"] = data["% Currency not in Circulation"].diff(year)    
data["Currency not in Circulation Growth Rate (%)"] = data["Base: Currency not in Circulation ($ Mil)"].pct_change(year) * 100    
data["Inflation (CPI)"] =  web.DataReader("CPIAUCSL", "fred", start, end).resample(freq).mean().pct_change(year).mul(100).dropna()
data["Inflation (PCE)"] = web.DataReader("PCEPILFE", "fred", start, end).resample(freq).mean().pct_change(year).mul(100).dropna()

data["Effective Federal Funds Rate Diff (%)"] = data["Effective Federal Funds Rate (%)"].diff(year)
data["1 Year Treasury Rate (%; diff)"] = data["1 Year Treasury Rate (%)"].diff(year)
data["2 Year Treasury Rate (%; diff)"] = data["2 Year Treasury Rate (%)"].diff(year)
data["10 Year Treasury Rate (%; diff)"] = data["10 Year Treasury Rate (%)"].diff(year)
data["30 Year Treasury Rate (%; diff)"] = data["30 Year Treasury Rate (%)"].diff(year)
data["Unemployment Rate Diff"] = data["Unemployment Rate"].diff(year)
data["Nominal GDP ($ Mil)"] = data["Nominal GDP ($ Bil)"].mul(1000)
data["Nominal GDP Growth Rate (%)"] = data["Nominal GDP ($ Bil)"].pct_change(year) * 100
data["Real GDP ($ Mil)"] = data["Real GDP ($ Bil)"].mul(1000)
data["Real GDP Growth Rate (%)"] = data["Real GDP ($ Bil)"].pct_change(year) * 100
data["Inflation (GDPDEF)"] = data["GDP Deflator"].pct_change(year) * 100
data["Real Currency in Circulation Growth Rate (%)"] = data["Currency in Circulation Growth Rate (%)"].sub(data["Inflation (GDPDEF)"])
data["Currency in Circulation Velocity"] = data["Nominal GDP ($ Mil)"].div(data["Base: Currency in Circulation ($ Mil)"])
data["Currency in Circulation % Change Velocity"] = data["Currency in Circulation Velocity"].pct_change(year)

data["Inflation Loss"]= data["Inflation (PCE)"].sub(inflation_target)
data["Unemployment Loss"]= data["Unemployment Rate"].sub(unemployment_target)
data["Inflation Loss Sq"]= data["Inflation (PCE)"].sub(inflation_target).pow(2)
data["Inflation Loss Sq"][data["Inflation Loss"] < 0] = data["Inflation Loss Sq"].mul(-1)

data["Unemployment Loss Sq"]= data["Unemployment Rate"].sub(unemployment_target).pow(2)
data["Unemployment Loss Sq"][data["Unemployment Loss"] < 0] = data["Unemployment Loss Sq"].mul(-1)



data["Inflation Loss Diff"]= data["Inflation Loss"].diff(year)
data["Unemployment Loss Diff"]= data["Unemployment Loss"].diff(year)
data["Inflation Loss Sq Diff"]= data["Inflation Loss Sq"].diff(year)
data["Unemployment Loss Sq Diff"]= data["Unemployment Loss Sq"].diff(year)



data["Linear Loss"] = data["Inflation Loss"].sub(data["Unemployment Loss"])
data["Loss Function"] = data["Inflation Loss Sq"].sub(data["Unemployment Loss Sq"])
data["Linear Loss Diff"] = data["Linear Loss"].diff(year)
data["Loss Function Diff"] = data["Loss Function"].diff(year)

data["Real 1 Year Treasury Rate"] = data["1 Year Treasury Rate (%)"].sub(data["Inflation (CPI)"])
data["Real 3 Month Treasury Rate"] = data["3 Month Treasury Rate (%)"].sub(data["Inflation (CPI)"])
data["Real 1 Month Treasury Rate"] = data["1 Month Treasury Rate (%)"].sub(data["Inflation (CPI)"])
data["Real Effective Federal Funds Rate"] = data['Effective Federal Funds Rate (%)'].sub(data["Inflation (CPI)"])

data["30 Year Minus 1 Year (%)"] = data["30 Year Treasury Rate (%)"].sub(data["1 Year Treasury Rate (%)"])
data["30 Year Minus 3 Month (%)"] = data["30 Year Treasury Rate (%)"].sub(data["3 Month Treasury Rate (%)"])
data["30 Year Minus 1 Month (%)"] = data["30 Year Treasury Rate (%)"].sub(data["1 Month Treasury Rate (%)"])
data["30 Year Minus Effective Federal Funds Rate"] = data["30 Year Treasury Rate (%)"].sub(data['Effective Federal Funds Rate (%)'])
data["10 Year Minus 2 Year (%)"] = data["10 Year Treasury Rate (%)"].sub(data["2 Year Treasury Rate (%)"])
data["10 Year Minus 1 Year (%)"] = data["10 Year Treasury Rate (%)"].sub(data["1 Year Treasury Rate (%)"])
data["10 Year Minus 3 Month (%)"] = data["10 Year Treasury Rate (%)"].sub(data["3 Month Treasury Rate (%)"])
data["10 Year Minus 1 Month (%)"] = data["10 Year Treasury Rate (%)"].sub(data["1 Month Treasury Rate (%)"])
data["10 Year Minus Effective Federal Funds Rate"] = data["10 Year Treasury Rate (%)"].sub(data['Effective Federal Funds Rate (%)'])

keys = list(data.keys())
keys = ["Date"] + keys
data["Date"] = data.index.astype(str)



C:\Users\JLCat\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [2]:
import pandas as pd
sheet_names = pd.ExcelFile('BEAGrossOutputbyindustryQuarterly.xlsx').sheet_names  # see all sheet names

info = pd.read_excel("BEAGrossOutputbyindustryQuarterly.xlsx", sheet_name = sheet_names[0], header = [3]).dropna(axis = 1)
key_names = {}
for ix in info.index:
    row = info.loc[ix] 
    key, val = row.values[0], row.values[1]
    key_names[key] = val
key_names



{'TGO101': 'Percent Changes in Chain-Type Quantity Indexes for Gross Output by Industry',
 'TGO103': 'Chain-Type Quantity Indexes for Gross Output by Industry',
 'TGO104': 'Chain-Type Price Indexes for Gross Output by Industry',
 'TGO105': 'Gross Output by Industry',
 'TGO106': 'Real Gross Output by Industry',
 'TGO107': 'Percent Changes in Chain-Type Price Indexes for Gross Output by Industry'}

In [3]:
sector_data = {}

for sheet_name, variable in key_names.items():
    sector_data[variable] = pd.read_excel("BEAGrossOutputbyindustryQuarterly.xlsx", 
                            index_col = [1],
                            sheet_name=sheet_name+"-Q", 
                            header = [7], parse_dates= True).T.iloc[2:].dropna(axis = 1)
    sector_data[variable].rename(index = {ix:ix.replace("Q","-") for ix in sector_data[variable].index},
                                inplace = True)
    sector_data[variable].rename(index = {ix:ix[:-1] + ("0" + str(int(ix[-1]) * 3))[-2:] for ix in sector_data[variable].index},
                                inplace = True)   
    sector_data[variable].index = pd.DatetimeIndex(sector_data[variable].index)
sector_data.keys()

dict_keys(['Percent Changes in Chain-Type Quantity Indexes for Gross Output by Industry', 'Chain-Type Quantity Indexes for Gross Output by Industry', 'Chain-Type Price Indexes for Gross Output by Industry', 'Gross Output by Industry', 'Real Gross Output by Industry', 'Percent Changes in Chain-Type Price Indexes for Gross Output by Industry'])

In [4]:
indices = ['Chain-Type Quantity Indexes for Gross Output by Industry',
           'Chain-Type Price Indexes for Gross Output by Industry', 
           'Gross Output by Industry', 
           'Real Gross Output by Industry']

layers = {}
for index in indices:
    data = sector_data[index]
    layers[index] = {0:[],
             1:{"Private industries":[], "Government":[]},
             2:{}}
    sector = "Private industries"
    for key in sector_data[index].keys():
        if key == "Government":
            sector = "Government"
        if key == key.strip():
            layers[index][0].append(key)
            subsector = key

        elif key[:2] == "  " and key[:3] != "   ":
            layers[index][1][layers[index][0][-1]].append(key)

#         elif key[:4] == "    " and key[:5] != "     ":
#             layers[index][2][layers[index][0][-1]].append(key)
            
layers    
#     elif "All industries" not in key:
#         layers[2][sector]
#         .append(layers[0][-1] + ": "+ layers[1][-1] + ": " + key)
private_and_public = layers['Chain-Type Quantity Indexes for Gross Output by Industry'][1]["Private industries"] + layers['Chain-Type Quantity Indexes for Gross Output by Industry'][1]["Government"]
private_and_public

['  Agriculture, forestry, fishing, and hunting',
 '  Mining',
 '  Utilities',
 '  Construction',
 '  Manufacturing',
 '  Wholesale trade',
 '  Retail trade',
 '  Transportation and warehousing',
 '  Information',
 '  Finance, insurance, real estate, rental, and leasing',
 '  Professional and business services',
 '  Educational services, health care, and social assistance',
 '  Arts, entertainment, recreation, accommodation, and food services',
 '  Other services, except government',
 '  Federal',
 '  State and local']

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
import pingouin

plt.rcParams.update({"font.size":24})
vals = private_and_public
for group in layers:

    pp = PdfPages(group + "LinePlots.pdf")
    df = sector_data[group]
    keys = layers[group][1]["Private industries"] + layers[group][1]["Government"]
    df = df[keys]
    i = 0

    for key in keys:
        color = "C" + str(i)
        color_df = df[[key]]
        other_keys = [k for k in keys if k != key]
        grey_df = df[other_keys]
        fig, ax = plt.subplots(figsize = (30,30))
        grey_df.plot.line(ax=ax, 
                          color = "k", 
                          alpha = .3, 
                          linewidth = 5, 
                          legend = False)
        for other_key in other_keys:
            plt.text(grey_df.index[-1], grey_df[other_key].iloc[-1], 
                     other_key, color = "k", alpha= .3)                
        color_df.plot.line(ax = ax, color = color, linewidth = 10, legend = False)
        plt.text(color_df.index[-1], color_df[key].iloc[-1], key, color = color, fontsize= 30)


        i += 1
        plt.title(group)
        pp.savefig(fig, bbox_inches = "tight")  
        plt.close()
    pp.close()
    pp = PdfPages(group + "ScatterPlots.pdf")

    df["Date"] = df.index.astype(str).str[:4].astype(int)    
    for key1 in keys:
        for key2 in keys:
            if key1 != key2:
                fig, ax = plt.subplots(figsize= (20,20))
                im = ax.scatter(x = df[key1], y = df[key2], c = df["Date"], 
                                cmap = "viridis", s = 50)
                ax.set_xlabel(key1)
                ax.set_ylabel(key2)
                cbar = plt.colorbar(im)
                cbar.ax.set_ylabel('Date')

                pp.savefig(fig, bbox_inches = "tight")
                plt.title(group)
#                 plt.show()
                plt.close()
    pp.close()
    corr_keys = list(df.keys())[:-1]
    for key in corr_keys:
        df[key] = df[key].astype(float)
    df.corr().dropna().to_csv(group + "CorrMatrix.csv")
    df.pcorr().dropna().to_csv(group + "PcorrMatrix.csv")

        
    
#         series.plot(ax = ax, logy = False, legend = False, linewidth = 10)
#         j = 0
#         for key in series.keys():
#             color = "C" + str(j)
#             plt.text(series.index[-1], series[key].iloc[-1], key, color = color)
#             j += 1
    

    
    
    
#     fig, ax = plt.subplots(figsize = (30,30))
#     i = 0
#     df = sector_data[group]
#     print(val)
# #         print(val)
#     keys = layers[group][1]
#     for key, vals in keys.items():
#         for val in vals:
#             series = df[val]
#             j = i % 10
#             series.plot(ax = ax, logy = False, legend = False, linewidth = 10)
#             plt.text(series.index[-1], series.iloc[-1], series.index.name, color = "C" + str(j))
#             i+=1
#     plt.title(group)
#     plt.show()
#     plt.close()

#     i = 0
#     for key, val in layers[group][1].items():
#         for series in val:
#             fig, ax = plt.subplots(figsize = (30,30))
#             for series2 in val:
#                 if series2.keys() != series.keys():                    
#                     series2.plot(ax = ax, logy = False, legend = False, color = "k", alpha = .3, linewidth = 5)
#                     plt.text(series2.index[-1], series2.iloc[-1], series2.keys()[0], color = "k", alpha = .4)
                
#             j = i % 10
#             color = "C" + str(j)
#             series.plot(ax = ax, logy = False, legend = False, color = color, linewidth = 10)
#             plt.text(series.index[-1], series.iloc[-1], series.keys()[0], color = color, size = 50)
            
            
#             i+=1
#             plt.title(group)
#             plt.show()
#             plt.close()

    
    

<ipython-input-5-42b9201b41d0>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = df.index.astype(str).str[:4].astype(int)
<ipython-input-5-42b9201b41d0>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[key].astype(float)
<ipython-input-5-42b9201b41d0>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
import pingouin
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
from matplotlib.patches import Rectangle
from pgmpy.estimators import PC
from pgmpy.base import DAG
import matplotlib.pyplot as plt
from matplotlib.patches import ArrowStyle
import networkx as nx
plt.rcParams.update({"font.size":20})

def graph_DAG(edges, df, title = ""):
    pcorr = df.pcorr()
    graph = nx.DiGraph()
    edge_labels = {}
    for edge in edges:
        edge_labels[edge] = str(round(pcorr[edge[0]].loc[edge[1]],2))
        
    graph.add_edges_from(edges)
    color_map = ["C7" for g in graph]

    fig, ax = plt.subplots(figsize = (30,30))
    graph.nodes()
    plt.tight_layout()
    pos = nx.spring_layout(graph, k = 5/(len(edge_labels.keys())**.5))

    plt.title(title, fontsize = 30)
    nx.draw_networkx(graph, pos, node_color=color_map, node_size = 1200,
                     with_labels=True,  arrows=True,
                     #font_color = "white",
                     font_size = 26, alpha = 1,
                     width = 1, edge_color = "C1",
                     arrowstyle=ArrowStyle("Fancy, head_length=3, head_width=1.5, tail_width=.1"), ax = ax)
    nx.draw_networkx_edge_labels(graph,pos,
                                 edge_labels=edge_labels,
                                 font_color='green',
                                 font_size=20)
    
def calculate_pcorr(data, keys):
    pcorr_df = data[keys]

    pcorr_dct = {}
    p_val_dct = {}
    for key1 in keys:
        p_val_dct[key1] = {}
        pcorr_dct[key1] = {}
        for key2 in keys:
            if key1 != key2:
                other_vars = [z for z in pcorr_df.keys() if z != key1 and z != key2 ]
                stats = pingouin.partial_corr(data=pcorr_df, 
                                              x = key1, 
                                              y = key2, 
                                              covars=other_vars)
                p_val_dct[key1][key2] = stats["p-val"].values[0]

                pcorr_dct[key1][key2] = pcorr_df.pcorr()[key1][key2]
            else:
                p_val_dct[key1][key2] = 1
                pcorr_dct[key1][key2] = 1

    p_val_df = pd.DataFrame(p_val_dct)
    pcorr_df = pd.DataFrame(pcorr_dct)
    return pcorr_df, p_val_df


for group in layers:
    pp = PdfPages(group + "corrPlots.pdf")
    df = sector_data[group]
    keys = layers[group][1]["Private industries"] + layers[group][1]["Government"]
    df = df[keys].pct_change(4)
    pcorr_df, p_val_df = calculate_pcorr(df, keys)

    plot_df = pcorr_df.dropna()
    plot_pvals_df = p_val_df.dropna()

    sig_vals = [.05, .01, .001]
    for key in plot_df:
        plot_df[key] = plot_df[key].round(3).astype(str)
        #plot_pvals_df[key] = plot_pvals_df[key].astype(str)
        for ix in plot_df.index:
            value = plot_df.loc[ix, key]
            p_val = plot_pvals_df.loc[ix, key]
            for sig_val in sig_vals:
                if p_val < sig_val: 
                    plot_df.loc[ix,key] = plot_df.loc[ix, key] + "*"

    plot_df = pcorr_df.dropna()

    plt.rcParams.update({'font.size': 20}) 

    # pcorr_df.style.use(p_val_df.style.applymap(bold_sig).export())
    fig, ax = plt.subplots(figsize = (50,50))
    mx = np.abs(pcorr_df.max().max())
    mn = np.abs(pcorr_df.min().min())
    max_val = max([mx,mn])
    min_val = max_val * -1
    pcorr_plot_df = plot_df.rename(columns = {key:key.replace("  ", "").replace(" ", "\n") for key in plot_df.keys()}, index = {ix:ix.replace("  ", "").replace(" ", "\n") for ix in plot_df.index})
    
    countries = plot_df.index
    keys = plot_df.keys()
    mp = sn.heatmap(pcorr_plot_df, annot = True, linewidth=1,cmap = "seismic", ax=ax, vmin=min_val, vmax=max_val)
    plt.title("Partial Correlation\n"+group, fontsize = 50)
    for i in range(len(keys)):
        for j in range(len(countries)):
            key = keys[i]
            country = countries[j]
            p_val = p_val_df.loc[country][key]
            lw = 0
            for k in range(len(sig_vals)):
                sig_val = sig_vals[k]
                if p_val <= sig_val:
                    lw = 3 * (k + 1) ** 2
    #         anchor = (lw / 2, lw / 2)
            mp.add_patch(Rectangle((i, j), 1, 1, fill=False, edgecolor='k', lw=lw))
    plt.show()
    plt.close()
    
    dag_df = df.dropna().rename(
        columns={
            col:col.replace("  ", "").replace(" ", "\n") for col in dag_df.keys()})
    variant = "parallel"
    sig = 0.0005
    ci_test = "pearsonr"
    keys = dag_df.keys()
    c = PC(dag_df)
    max_cond_vars = len(keys) - 2
    model = c.estimate(return_type = "dag",variant= variant, 
                       significance_level = sig, 
                       max_cond_vars = max_cond_vars, ci_test = ci_test)
    edges = model.edges()
    graph_DAG(edges, dag_df, title = group)    
# pcorr_df[p_val_df["General Outcome"] < sig]#.style.apply(highlight)
# pcorrsig[p_val_df["General Outcome"] < sig].to_csv("StatisticallySignificantOutcomesbyCountry.csv")
# p_val_df[p_val_df["General Outcome"] < sig].to_csv("StatisticalSignificancebyCountry.csv")

In [ ]:
dag_df

In [ ]:
# import pandas_datareader.data as web
# start = datetime.datetime(2001, 12, 1)
# end = datetime.datetime.today()
# x =  web.DataReader("CPIAUCSL", "fred", start, end).resample(freq).mean().pct_change(year).mul(100).dropna()
# x

In [ ]:
import yfinance as yfin
yfin.pdr_override()
data["S&P"]= web.get_data_yahoo(ticker, start = start, end = end).resample(freq).mean()["Close"].iloc[:-2]
data["S&P Growth Rate (%)"] = data["S&P"].pct_change(year)
data["S&P Growth Rate Change Diff (%)"] = data["S&P Growth Rate (%)"].diff(year)
data["Real S&P Growth Rate (%)"] = data["S&P Growth Rate (%)"].sub(data["Inflation (CPI)"])
data["VIX Diff"] = data["VIX"].diff(year)
data

In [ ]:
#fedProject
# . . . 
data["Balance Sheet: Direct Lending and Central Bank Liquidity Swaps"] =\
    data["Balance Sheet: Central Bank Liquidity Swaps ($ Mil)"].add(
    data["Balance Sheet: Direct Lending ($ Mil)"])
data["Balance Sheet: Other Securities"] = data["Balance Sheet: Securities Held Outright ($ Mil)"].sub(
    data["Balance Sheet: U.S. Treasuries Held Outright ($ Mil)"]).sub(
    data["Balance Sheet: Mortgage-Backed Securities ($ Mil)"])
data["Balance Sheet: Other Assets"] = data["Balance Sheet: Total Assets ($ Mil)"].sub(
    data["Balance Sheet: Securities Held Outright ($ Mil)"]).sub(
    data["Balance Sheet: Direct Lending and Central Bank Liquidity Swaps"]).sub(
    data["Balance Sheet: Repos ($ Mil)"]).sub(
    data["Balance Sheet: Unamortized Security Premiums ($ Mil)"])
data["Balance Sheet: Other Deposits ($ Mil)"] = data["Balance Sheet: All Deposits ($ Mil)"].sub(
    data["Balance Sheet: U.S. Treasury General Account ($ Mil)"]).sub(
    data["Balance Sheet: Deposits from Dep. Institutions ($ Mil)"])
data["Balance Sheet: Other Liabilities"]= data["Balance Sheet: Total Liabilities ($ Mil)"].sub(
    data["Balance Sheet: Federal Reserve Notes Outstanding ($ Mil)"]).sub(
    data["Balance Sheet: U.S. Treasury General Account ($ Mil)"]).sub(
    data["Balance Sheet: Deposits from Dep. Institutions ($ Mil)"]).sub(
    data["Balance Sheet: Other Deposits ($ Mil)"]).sub(
    data["Balance Sheet: Reverse Repos ($ Mil)"])


In [ ]:
plt.rcParams.update({"font.size":32})
interest_vars = ["Effective Federal Funds Rate (%)",
                   "Interest on Reserves (%)",
                   "1 Month Treasury Rate (%)"]
fig, ax = plt.subplots(figsize = (30,20))
data[interest_vars].plot.line(legend=True, linewidth = 3, ax = ax)
y_vals = ax.get_yticks()
ax.set_yticklabels([str(round(y,2))+ "%" for y in y_vals])  
ax.set_title("Short Term Interest Rates", fontsize = 48)
data[["Date"] + interest_vars].to_json("shortTermRates.json", orient="records")

In [ ]:
data = data.rename(columns = {
    key: key.replace("Balance Sheet: ", "").replace(" ($ Mil)", "").replace("Base: ","") for key in data.keys()})
keys = list(data.keys())
interest_rates = ["Effective Federal Funds Rate (%)",
                   "Interest on Reserves (%)",
                   "1 Month Treasury Rate (%)"]
plot_data = data.copy()
for key, val in data.items():
    if key not in interest_rates:
        try:
            plot_data[key] = val.div(10**6)
        except:
            continue

In [ ]:
account_vars = ["U.S. Treasuries Held Outright",
               "Mortgage-Backed Securities",
               "Other Securities",
               "Direct Lending and Central Bank Liquidity Swaps",
               "Repos",
               "Unamortized Security Premiums",
               "Other Assets"]
figsize= (36,18)
fig, ax = plt.subplots(figsize = figsize)
plot_data[account_vars].plot.area(stacked = True, linewidth = 3,
                            ax = ax)
# change y vals from mil to tril
total_var = "Total Assets"
plot_data[total_var].plot.line(linewidth = 1, 
                             ax = ax, c = "k",
                             label = total_var, ls = "--")
plt.xticks(rotation = 90)
ax.legend(loc=2, ncol = 2)
ax.set_ylabel("$ Trillion", fontsize = 40)
ax.set_title("Federal Reserve Balance Sheet: Assets", fontsize = 50)
plot_data[[total_var] + account_vars].to_csv("FederalReserveAssets.csv")
plot_data[["Date"] + account_vars].to_json("fedAssets.json", orient = "records")

In [ ]:
account_vars = ["Federal Reserve Notes Outstanding",
                "U.S. Treasury General Account",
                "Deposits from Dep. Institutions",
                "Other Deposits",
                "Reverse Repos",
                "Other Liabilities"]
total_var = "Total Liabilities"

plot_stacked_lines(
    plot_data,
    account_vars, linewidth = 1,
    total_var = "Total Liabilities")


In [ ]:
figsize= (36,18)
fig, ax = plt.subplots(figsize = figsize)
plot_data[account_vars].plot.area(stacked = True, linewidth = 3,
                            ax = ax)
# change y vals from mil to tril
total_var = "Total Liabilities"
plot_data[total_var].plot.line(linewidth = 2, 
                             ax = ax, c = "k",
                             label = total_var, ls = "--")
plt.xticks(rotation = 90)
ax.legend(loc=2, ncol = 2)
ax.set_ylabel("$ Trillion", fontsize = 40)
ax.set_title("Federal Reserve Balance Sheet: Liabilities", fontsize = 50)
plot_data[[total_var] + account_vars].to_csv("FederalReserveLiabilities.csv")

In [ ]:
plt.rcParams.update({"font.size":32})
df = plot_data.copy()
for key in account_vars:
    df[key] = df[key].div(df["Total Liabilities"])
figsize= (36,18)
fig, ax = plt.subplots(figsize = figsize)
df[account_vars].plot.area(stacked = True, linewidth = 3,
                            ax = ax)
plt.xticks(rotation= 90, fontsize = 45)
ax.set_yticklabels([str(int(val * 100)) + "% " for val in ax.get_yticks()], fontsize = 45)

#plt.yticks([int(tick * 100) for tick in ax.get_yticks() if tick <1.01])
plt.title("Liabilities as a Proportion of the\nFederal Reserve's Balance Sheet", fontsize =45)
plt.show()


In [ ]:
plot_data["Net Effect on Overnight Lending Market"] =\
    plot_data["Repos"].sub(plot_data["Reverse Repos"])
overnight_vars = ["Repos",
                  "Reverse Repos",
                  "Net Effect on Overnight Lending Market"]
fig, ax = plt.subplots(figsize = (30,20))
plot_data[overnight_vars].plot.line(legend=True, linewidth = 3, ax = ax)
ax.set_ylabel("$ Trillion", fontsize = 40)
ax.set_title("Federal Reserve Activity\nin Overnight Lending Market", fontsize = 48)
plot_data[["Date"] + overnight_vars].to_json("overnightLending.json", orient="records")


In [ ]:
data["Total Liabilities Growth Rate (%)"] = data["Total Liabilities"].pct_change(year)
data["Total Assets Growth Rate (%)"] = data["Total Assets"].pct_change(year)
data["Total Liabilities / Currency in Circulation"] = data["Total Liabilities"].div(data["Currency in Circulation ($ Bil)"].mul(1000))
data["Total Assets / Currency in Circulation"] = data["Total Assets"].div(data["Currency in Circulation ($ Bil)"].mul(1000))
data["Currency in Circulation / Total Assets"] = data["Currency in Circulation ($ Bil)"].mul(1000).div(data["Total Assets"])
data["Currency in Circulation / Total Assets Diff"] = data["Currency in Circulation / Total Assets"].diff(year)
data["Currency in Circulation / Total Liabilities"] = data["Currency in Circulation ($ Bil)"].mul(1000).div(data["Total Liabilities"])
data["Currency in Circulation / Total Liabilities Diff"] = data["Currency in Circulation / Total Liabilities"].diff(year)

data["Log Total Liabilities"] = np.log(data["Total Liabilities"])
data["Log Total Assets"] = np.log(data["Total Assets"])
#data["Currency in Circulation / Total Liabilities %"] = data["Currency in Circulation / Total Liabilities"].pct_change()


In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
data

In [ ]:
import pingouin
plot_vars = ["Inflation (PCE)",
             "Unemployment Rate",
             "Inflation Loss",
             "Unemployment Loss",
             "Linear Loss",
             "Loss Function",
             "Effective Federal Funds Rate (%)",
             "Currency in Circulation Growth Rate (%)",
             "Currency in Circulation / Total Assets",
             "Total Assets Growth Rate (%)"]
#data[plot_vars].describe().T.to_excel("C:\\Users\\JLCat\\OneDrive\\Documents\\For NDSU\\Projects\\Sound Money Project\\Frederal Reserve QE Framework\\DescriptionStatistics.xls")
#data[plot_vars].describe().corr()
#data[plot_vars].describe().pcorr()

In [ ]:
plot_vars = [#Currency in Circulation Growth Rate (%)",
             #"Currency not in Circulation Growth Rate (%)",
             #"5 Year Forward Rate",
             #"Inflation (CPI)",
             #"Unemployment Rate Diff",
             #"Loss Function Diff",
            #"Unemployment Loss Diff",
            #"Inflation Loss Diff",
            #"Linear Loss Diff",
            #"Unemployment Loss Sq Diff",
            #"Inflation Loss Sq Diff",
            "Linear Loss Diff",
    #"Currency in Circulation % Change Velocity",
             #"Nominal GDP Growth Rate (%)",
             #"Real GDP Growth Rate (%)",
             "Effective Federal Funds Rate Diff (%)",
             #"30 Year Treasury Rate (%)",
             #"1 Month Treasury Rate (%)",
             #"S&P Growth Rate (%)",
             "Currency in Circulation / Total Assets Diff",
             #Total Assets Growth Rate (%)",
             "VIX Diff"
]
dag_df = data[plot_vars].dropna().rename(columns={key: key.replace(" ", "\n") for key in plot_vars}).loc["2008-10-01":"2020-08-01"]
dag_df.pcorr().round(2)

In [ ]:

    
plot_vars_dct = {0: ["Loss Function Diff",
                     "Effective Federal Funds Rate Diff (%)",
                     "Currency in Circulation / Total Assets Diff"],
#                     "VIX Diff"],
                 1:["Currency in Circulation Growth Rate (%)",
                    "Loss Function Diff",
                    "Effective Federal Funds Rate Diff (%)",
                    "Total Assets Growth Rate (%)"],
 #                   "VIX Diff"],
                 2:["Unemployment Loss Sq Diff",
                    "Inflation Loss Sq Diff",
                    "Effective Federal Funds Rate Diff (%)",
                    "Currency in Circulation / Total Assets Diff"],
#                    "VIX Diff"],
                 3:["Currency in Circulation Growth Rate (%)",
                    "Unemployment Loss Sq Diff",
                    "Inflation Loss Sq Diff",
                    "Effective Federal Funds Rate Diff (%)",
                    "Total Assets Growth Rate (%)"],
                    #"VIX Diff"]
}

sig = 0.01
variant = "parallel"
ci_test = "pearsonr"
for key, plot_vars in plot_vars_dct.items():
    dfs = {"2008-10-31 to 2020-07-31": data[plot_vars].dropna().rename(columns={key: key.replace(" ", "\n") for key in plot_vars}).loc["2008-10-31":"2020-08-31"],
           "2003-12-31 to 2008-09-30": data[plot_vars].dropna().rename(columns={key: key.replace(" ", "\n") for key in plot_vars}).loc[:"2008-09-30"]}
           
    for dates, dag_df in dfs.items():

        keys = dag_df.keys()
        c = PC(dag_df.dropna())
        max_cond_vars = len(keys) - 2
#         print(dag_df.index)

        model = c.estimate(return_type = "dag",variant= variant, 
                           significance_level = sig, 
                           max_cond_vars = max_cond_vars, ci_test = ci_test)
        edges = model.edges()
        pcorr = dag_df.pcorr()
        weights = {}
#         for edge in edges:
#             print(edge, ":",pcorr[edge[0]].loc[edge[1]])
#         skel, sep_sets = c.build_skeleton(variant = variant, ci_test = ci_test, significance_level = sig, 
#                            max_cond_vars = max_cond_vars)


        graph_DAG(edges, dag_df, title = dates)

In [ ]:
from pgmpy.estimators import PC
from pgmpy.base import DAG

keys = dag_df.keys()
c = PC(dag_df.dropna())
max_cond_vars = len(keys) - 2

sig = 0.05
variant = "parallel"
ci_test = "pearsonr"
model = c.estimate(return_type = "dag",variant= variant, 
                   significance_level = sig, 
                   max_cond_vars = max_cond_vars, ci_test = ci_test)

edges = model.edges()
pcorr = dag_df.pcorr()
weights = {}
for edge in edges:
    print(edge, ":",pcorr[edge[0]].loc[edge[1]])
skel, sep_sets = c.build_skeleton(variant = variant, ci_test = ci_test, significance_level = 0.01, 
                   max_cond_vars = max_cond_vars)
sep_sets

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import ArrowStyle
import networkx as nx
plt.rcParams.update({"font.size":20})

def graph_DAG(edges, df, title = ""):
    pcorr = df.pcorr()
    graph = nx.DiGraph()
    edge_labels = {}
    for edge in edges:
        edge_labels[edge] = str(round(pcorr[edge[0]].loc[edge[1]],2))
        
    graph.add_edges_from(edges)
    color_map = ["C0" for g in graph]

    fig, ax = plt.subplots(figsize = (20,20))
    graph.nodes()
    plt.tight_layout()
    pos = nx.spring_layout(graph)#, k = 5/(len(sig_corr.keys())**.5))

    plt.title(title, fontsize = 30)
    nx.draw_networkx(graph, pos, node_color=color_map, node_size = 1200,
                     with_labels=True,  arrows=True,
                     #font_color = "white",
                     font_size = 26, alpha = 1,
                     width = 1, edge_color = "C1",
                     arrowstyle=ArrowStyle("Fancy, head_length=3, head_width=1.5, tail_width=.1"), ax = ax)
    nx.draw_networkx_edge_labels(graph,pos,
                                 edge_labels=edge_labels,
                                 font_color='green',
                                 font_size=20)

graph_DAG(edges, dag_df)

In [ ]:
from pgmpy.estimators import PC
from pgmpy.base import DAG

keys = dag_df.keys()
c = PC(dag_df.dropna())
max_cond_vars = len(keys) - 2

variant = "parallel"
ci_test = "pearsonr"
model = c.estimate(return_type = "dag",variant= variant, 
                   significance_level = sig, 
                   max_cond_vars = max_cond_vars, ci_test = ci_test)

edges = model.edges()
pcorr = dag_df.pcorr()
weights = {}
for edge in edges:
    print(edge, ":",pcorr[edge[0]].loc[edge[1]])
skel, sep_sets = c.build_skeleton(variant = variant, ci_test = ci_test, significance_level = 0.01, 
                   max_cond_vars = max_cond_vars)
sep_sets

In [ ]:
graph_DAG(edges, dag_df)

In [ ]:
from pgmpy.estimators import PC
from pgmpy.base import DAG

keys = dag_df.keys()
c = PC(dag_df.dropna())
max_cond_vars = len(keys) - 2

variant = "parallel"
ci_test = "pearsonr"
model = c.estimate(return_type = "dag",variant= variant, 
                   significance_level = sig, 
                   max_cond_vars = max_cond_vars, ci_test = ci_test)

edges = model.edges()
pcorr = dag_df.pcorr()
weights = {}
for edge in edges:
    print(edge, ":",pcorr[edge[0]].loc[edge[1]])
skel, sep_sets = c.build_skeleton(variant = variant, ci_test = ci_test, significance_level = 0.01, 
                   max_cond_vars = max_cond_vars)
sep_sets

In [ ]:
graph_DAG(edges, dag_df)

In [ ]:
from pgmpy.estimators import PC
from pgmpy.base import DAG

keys = dag_df.keys()
c = PC(dag_df.dropna())
max_cond_vars = len(keys) - 2

variant = "parallel"
ci_test = "pearsonr"
model = c.estimate(return_type = "dag",variant= variant, 
                   significance_level = sig, 
                   max_cond_vars = max_cond_vars, ci_test = ci_test)

edges = model.edges()
pcorr = dag_df.pcorr()
weights = {}
for edge in edges:
    print(edge, ":",pcorr[edge[0]].loc[edge[1]])
skel, sep_sets = c.build_skeleton(variant = variant, ci_test = ci_test, significance_level = 0.01, 
                   max_cond_vars = max_cond_vars)
sep_sets

In [ ]:
for key in sep_sets:
    print(list(sep_sets[key]))

In [ ]:
graph_DAG(edges, dag_df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import ArrowStyle
import networkx as nx
plt.rcParams.update({"font.size":20})

def graph_DAG(edges, df,sep_sets, title = ""):
    graph = nx.DiGraph()
    edge_labels = {}
    pcorr = df.pcorr()
    for edge in sep_sets:
        sep_set = list(sep_sets[edge])
        key = list(edge)
#        pcorr = df[key + sep_set].pcorr()
        edge_labels[edge] = str(round(pcorr[key[0]].loc[key[1]],2))
    print(edge_labels)
    graph.add_edges_from(sep_sets.keys())
    color_map = ["C0" for g in graph]

    fig, ax = plt.subplots(figsize = (20,20))
    graph.nodes()
    plt.tight_layout()
    pos = nx.spring_layout(graph)#, k = 5/(len(sig_corr.keys())**.5))

    plt.title(title, fontsize = 30)
    nx.draw_networkx(graph, pos, node_color=color_map, node_size = 1200,
                     with_labels=True,  arrows=True,
                     #font_color = "white",
                     font_size = 26, alpha = 1,
                     width = 1, edge_color = "C1",
                     arrowstyle=ArrowStyle("Fancy, head_length=3, head_width=1.5, tail_width=.1"), ax = ax)
    nx.draw_networkx_edge_labels(graph,pos,
                                 edge_labels=edge_labels,
                                 font_color='green',
                                 font_size=20)

graph_DAG(edges, dag_df, sep_sets)

In [ ]:
dag_df

In [ ]:
figsize = (20,12)
fig,ax = plt.subplots(figsize = figsize)

ax2 = ax.twinx()
ax3 = ax.twinx()
data[["Currency in Circulation / Total Assets"]].plot(legend = False, 
                                                      linewidth = 5, 
                                                      alpha = .8,
                                                      ax = ax)
data[["Effective Federal Funds Rate (%)"]].plot(legend = False, 
                                                      linewidth = 5, 
                                                      alpha = .8,
                                                      ax = ax2,
                                                        c = "C1")
data[["Linear Loss"]].iloc[:-1].plot(legend = False, 
                                                      linewidth = 5, 
                                                      alpha = .1,
                                                      ax = ax3,
                                                        c = "k")

ax.set_yticklabels([str(int(val * 100)) + "%" for val in ax.get_yticks()], c = "C0")
ax2.set_yticklabels([str(int(val)) + "%" for val in ax2.get_yticks()], c = "C1")
ax3.set_yticklabels(ax3.get_yticks(), alpha = .2, c = "k")
ax3.set_yticks([])

ax3.axhline(0, c = "k", alpha = .1, linewidth = 5, ls = "--")

plt.axvline(datetime.datetime(2008,10,1), c = "k", ls = "--", alpha = .9)
ax.text(datetime.datetime(2013,3,1), .42, "Currency in Circulation\n/  \nTotal Assets", fontsize = 25, c = "C0", 
        ha = "center")
ax.text(datetime.datetime(2018,5,1), .58, "Effective\nFederal\nFunds\nRate", fontsize = 25, c = "C1", 
        ha = "center")
ax.text(datetime.datetime(2014,6,1), .78, "Linear\nLoss\nFunction", fontsize = 25, c = "k", alpha = .3, 
        ha = "center")
ax.text(datetime.datetime(2002,10,1), .955, "0", fontsize = 30, c = "k", alpha = .3, 
        ha = "right")
ax.text(datetime.datetime(2022,1,1), .955, "0 ", fontsize = 30, c = "k", alpha = .3, 
        ha = "right")

In [ ]:
fig, ax = plt.subplots(figsize = (30,15))
data[["Linear Loss Diff", "Total Assets Growth Rate (%)"]].plot.scatter(x = "Linear Loss Diff", y = "Total Assets Growth Rate (%)", ax = ax)
plt.xticks(rotation=90)


In [ ]:
fig, ax = plt.subplots(figsize = (30,15))
data.iloc[:-1].plot.scatter(x = "Linear Loss", y = "Currency in Circulation / Total Assets", ax = ax)
plt.xticks(rotation=90)
